# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tuktoyaktuk,75,CA,1635622816,93,69.4541,-133.0374,17.60,9.22
1,Bethel,90,US,1635622716,91,41.3712,-73.4140,62.73,8.05
2,Isangel,0,VU,1635622817,75,-19.5500,169.2667,77.40,4.50
3,Punta Arenas,0,CL,1635622711,43,-53.1500,-70.9167,51.91,18.41
4,Vaini,40,TO,1635622818,88,-21.2000,-175.2000,75.36,9.22
...,...,...,...,...,...,...,...,...,...
555,Lorengau,83,PG,1635623081,79,-2.0226,147.2712,81.63,6.78
556,Kavaratti,93,IN,1635623081,72,10.5669,72.6420,81.97,2.84
557,Henties Bay,0,NaN,1635623082,89,-22.1160,14.2845,57.85,4.61
558,Berdigestyakh,69,RU,1635623083,99,62.0989,126.6992,2.17,4.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,Isangel,0,VU,1635622817,75,-19.5500,169.2667,77.40,4.50
118,Tsabong,0,BW,1635622870,20,-26.0217,22.4010,73.80,6.64
165,Maxixe,0,MZ,1635622891,78,-23.8597,35.3472,71.42,9.22
237,Jalu,0,LY,1635622659,40,29.0331,21.5482,70.14,7.67
254,Bhag,0,PK,1635622936,14,29.0415,67.8239,77.25,4.90
261,Mocuba,0,MZ,1635622939,69,-16.8392,36.9856,70.20,4.76
306,Karratha,0,AU,1635622957,70,-20.7377,116.8463,72.46,6.40
372,Inhambane,0,MZ,1635622750,78,-23.8650,35.3833,71.58,9.22
469,Redwater,0,US,1635623035,47,33.3582,-94.2544,71.49,8.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Isangel,VU,-19.5500,169.2667,
118,Tsabong,BW,-26.0217,22.4010,
165,Maxixe,MZ,-23.8597,35.3472,
237,Jalu,LY,29.0331,21.5482,
254,Bhag,PK,29.0415,67.8239,
261,Mocuba,MZ,-16.8392,36.9856,
306,Karratha,AU,-20.7377,116.8463,
372,Inhambane,MZ,-23.8650,35.3833,
469,Redwater,US,33.3582,-94.2544,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 2: Isangel.
Closest hotel in Isangel is Amutia Resort.
------------
Retrieving Results for Index 118: Tsabong.
Closest hotel in Tsabong is Batshweneng Guest House.
------------
Retrieving Results for Index 165: Maxixe.
Closest hotel in Maxixe is Hotel Casa Do Capitão.
------------
Retrieving Results for Index 237: Jalu.
Closest hotel in Jalu is Thuraya Tourist Hotel.
------------
Retrieving Results for Index 254: Bhag.
Closest hotel in Bhag is Bukhari hotl.
------------
Retrieving Results for Index 261: Mocuba.
Closest hotel in Mocuba is Aschel guest house.
------------
Retrieving Results for Index 306: Karratha.
Closest hotel in Karratha is Best Western Plus The Ranges Karratha.
------------
Retrieving Results for Index 372: Inhambane.
Closest hotel in Inhambane is Hotel Casa Do Capitão.
------------
Retrieving Results for Index 469: Redwater.
Missing field/result... skipping.
------------
-------End of Search-------


In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Isangel,VU,-19.5500,169.2667,Amutia Resort
118,Tsabong,BW,-26.0217,22.4010,Batshweneng Guest House
165,Maxixe,MZ,-23.8597,35.3472,Hotel Casa Do Capitão
237,Jalu,LY,29.0331,21.5482,Thuraya Tourist Hotel
254,Bhag,PK,29.0415,67.8239,Bukhari hotl
261,Mocuba,MZ,-16.8392,36.9856,Aschel guest house
306,Karratha,AU,-20.7377,116.8463,Best Western Plus The Ranges Karratha
372,Inhambane,MZ,-23.8650,35.3833,Hotel Casa Do Capitão
469,Redwater,US,33.3582,-94.2544,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))